# Clinical Note Synopsis & Preventive Health Recommendations

In [15]:
import os
import openai
import webbrowser
import datetime 
from datetime import datetime
import json
import csv
import pandas as pd

import ast 
import subprocess
from datetime import datetime

from json import loads, dumps
from CSV_to_JSON_or_DICT import CSV_to_DICT


from chatGPT_utilities import setup_OpenAI_API
from chatGPT_utilities import json_to_dict
from chatGPT_utilities import pp_json
from chatGPT_utilities import get_completion
from chatGPT_utilities import get_completion_from_messages_FULL
from chatGPT_utilities import get_completion_from_messages
from chatGPT_utilities import read_file
from chatGPT_utilities import token_usage
from chatGPT_utilities import print_token_usage
from chatGPT_utilities import openai_object_to_dict
from chatGPT_utilities import CSV_to_DICT

openai.api_key = setup_OpenAI_API()
starting_directory = os.getcwd()


def replace_with_bold_red(string, substring):
    """
    : Replace substring with BOLD RED in the string
    """
    string = string.replace(substring,'\033[91m'+\
                                     '\033[1m' + '\n\n' + substring + '\033[0m') 
    return string

def write_to_file(file_name,patient_note,response):
    file = open(file_name + '.txt' ,'w') 
    now = datetime.now()

    formatted_date = now.strftime("%a %b %d at %H:%M")
    print(f'{formatted_date}')
    file.write('Created on: ' + formatted_date + '\n'*2 )
    
    file.write(patient_note + '\n'*2 )
    file.write(response + '\n'*2 )
    print(f'{file_name} was created in {os.getcwd()}')
    file.close() 
    return


def capitalize_all_letters(string):
    capitalized_string = ""
    for char in string:
        if char.isalpha():
            capitalized_string += char.upper()
        else:
            capitalized_string += char
    return capitalized_string



url_chatGPT_usage = 'https://platform.openai.com/account/usage'
_ = webbrowser.open_new(url_chatGPT_usage)

starting_directory /Users/michael/Coding
Directory:/Users/michael/Coding


# Read Clinical Note
- Extract last clinical note

In [77]:
[i for i in os.listdir() if i.endswith('txt')]
print('Current Directory:',os.getcwd())

Current Directory: /Users/michael/Coding


In [119]:
def color_code_clinic_note(pt_note,list_patterns):
    patient_note_for_display = pt_note
    for pattern in list_patterns:
        patient_note_for_display = patient_note_for_display.replace(pattern,
                                                                    '\033[91m\033[1m' + pattern + '\033[0m')
    return patient_note_for_display

In [120]:
dir_patient_notes = 'Sample_Patient_Notes'
file_name = 'Maria_Sanchez.txt'


patient_note = read_file(os.path.join(dir_patient_notes,
                                     file_name))

# print(patient_note)
list_patterns = ['Patient Information','Name','Age','Ethnicity',
                'Gender','Reason for Visit',
                'History of Present Illness',
                'Medical History','Family History','Social History',
                'Review of Systems','Physical Examination',
                'Assessment and Plan']

patient_note_for_display = color_code_clinic_note(patient_note,list_patterns)
print(patient_note_for_display)


Patient Information:
Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic
Address : Sacramento CA

Reason for Visit: Annual Physical

History of Present Illness:
The patient, a 67-year-old white female, presents today for her annual physical examination. She has not reported any specific complaints or concerns at this time.

Medical History:
The patient has a history of hypertension, which is well-controlled with enalapril and Hydralazine
She also has a history of hyperlipidemia and is on a simvastatin to manage her cholesterol levels.
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.


Family History:
The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Her mother is alive and in good health. Mother had colon cancer but is currently in remission.

Social History:
The patient smokes 1/2 ppd and has a glass of wine most evenings. She is retired and leads

In [112]:
# dir_patient_notes = 'Sample_Patient_Notes'
# file_name = 'Maria_Sanchez.txt'
# file_name = 'ENT_Patient_no_assesment.txt'


# patient_note = read_file(os.path.join(dir_patient_notes,
#                                      file_name))


# patient_note_original = patient_note

# patient_note_cap_version = patient_note_original
# list_substrings = ['Patient Information','Reason for Visit','Medical History',
#                   'Family History', 'Social History','Review of Systems',
#                   'Physical Examination','Assessment and Plan:']
# for substring in list_substrings:
#     patient_note_cap_version  
#     capitalized_string = capitalize_all_letters(substring)
#     patient_note_cap_version = patient_note_cap_version.replace(substring,capitalized_string)
# # print('\n'*5 ) 
# # print(patient_note_cap_version)


# list_substrings = ['Patient Information','Reason for Visit','Medical History','History of Present Illness',
#                   'Family History', 'Social History','Review of Systems',
#                   'Physical Examination','Assessment and Plan:']
# for substring in list_substrings:
#     index = patient_note.find(substring)
#     patient_note  = replace_with_bold_red(patient_note,  substring)
# print(patient_note)

In [79]:
_ = subprocess.call(['open',   os.path.join('Sample_Patient_Notes',file_name) ])

# Identify Medical Considerations based on Clinical Note
## Determine 5-7 medical considerations (one or two words in length) 


In [113]:

prompt = f"""
Your task is to extract relevant information from \ 
a clinical note. 


Determine 5-7 medical considerations that are being discussed in the \
clinical note below, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a numbered list of items, formatted as follows:
MEDICAL CONSIDERATIONS:
1. 
2. 

N. 

Review: ```{patient_note}```
"""
response = get_completion(prompt)
print(response)

MEDICAL CONSIDERATIONS:
1. Hypertension
2. Hyperlipidemia
3. Cardiovascular disease (family history)
4. Smoking
5. Alcohol consumption
6. Routine laboratory investigations
7. Medication management


# Create Assessment and Plan
- Read note
- Acting as a PCP, extract information
- Produce an assessent and plan
- Include relevant preventive health recommendations

In [72]:
medical_specialist = 'primary care physician'
prompt = f"""
Acting as a {medical_specialist}, your task is to extract relevant information from \ 
a clinical note to produce an assessent and plan for the patient.


Format your response as follows:
Patient Age:
Patient Gender:
CURRENT MEDICATIONS:
1.
2.

N.

MEDICAL CONSIDERATIONS:
1. Medical Concern
    Assessment : 
    Plan       :

2. Medical Concern
    Assessment : 
    Plan       :
    
N. Medical Concern
    Assessment : 
    Plan       :
    
    
Preventive Care Recommendations:
    a.
    b.

Preventive Care Recommendations should include any recommendations about 
screening for cancer  . 
List any screening recommendations under 'Preventive Care Recommendations' 
and do not include them as a Medical Consideration.
Be sure to mention Education regarding alcohol and tobacco in the preventive care recommendations, if applicable


Review: ```{patient_note}```
"""

In [70]:
    
# def extract_info_as_a(physician):
#     prompt = f"""
#     Acting as a {medical_specialist}, your task is to extract relevant information from \ 
#     a clinical note to produce an assessent and plan for the patient.


#     Format your response as follows:
#     Patient Age:
#     Patient Gender:
#     CURRENT MEDICATIONS:
#     1.
#     2.

#     N.

#     MEDICAL CONSIDERATIONS:
#     1. Medical Concern
#         Assessment : 
#         Plan       :

#     2. Medical Concern
#         Assessment : 
#         Plan       :

#     N. Medical Concern
#         Assessment : 
#         Plan       :


#     Preventive Care Recommendations:
#         a.
#         b.

#     Preventive Care Recommendations should include any recommendations about 
#     screening for cancer  . 
#     List any screening recommendations under 'Preventive Care Recommendations' 
#     and do not include them as a Medical Consideration.
#     Be sure to mention Education regarding alcohol and tobacco in the preventive care recommendations, if applicable
    
#     Do not refer patient to a doctor or specialist that is in the same specialty as a {medical_specialist}.

#     Review: ```{patient_note}```
#     """
#     return prompt

In [73]:
# medical_specialist = 'primary care physician'
# the_prompt = extract_info_as_a(medical_specialist)
# print(the_prompt.split('Review: ')[0])
assessment = get_completion(prompt)
print(assessment)

Patient Age: 67
Patient Gender: Female

CURRENT MEDICATIONS:
1. Enalapril
2. Hydralazine
3. Simvastatin

MEDICAL CONSIDERATIONS:
1. Hypertension
    Assessment: Well-controlled with current medications
    Plan: Continue current medications and monitor blood pressure regularly

2. Hyperlipidemia
    Assessment: Well-managed with simvastatin
    Plan: Continue simvastatin and monitor lipid levels regularly

Preventive Care Recommendations:
    a. Annual lipid panel to monitor hyperlipidemia
    b. Colon cancer screening due to family history (mother had colon cancer)

Note: Education regarding smoking cessation and alcohol consumption should be provided to the patient during the follow-up visit.


In [114]:


start_time = datetime.now() 

messages = [{"role": "user", "content": prompt}]
response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
    # print(response)

# response = get_completion(prompt)
end_time = datetime.now() 
print(response, '\n'*2)
print('end_time  :', end_time.strftime("%H:%M:%S"))
elapsed_time = end_time - start_time
print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
cost = print_token_usage(response_details)
print(f'Cost to run query: ${cost}')

MEDICAL CONSIDERATIONS:
1. Hypertension
2. Hyperlipidemia
3. Cardiovascular disease (family history)
4. Smoking
5. Alcohol consumption
6. Routine laboratory investigations
7. Medication management 


end_time  : 13:49:28
Elapsed_time: 2.713 seconds
Cost to run query is based on token usage, which is $0.002 / 1K tokens


Token Usage:


completion_tokens: 49
prompt_tokens    : 777
total_tokens     : 826
Cost to run query: $0.002


In [115]:
# file_name = 'Maria Sanchez_Friday'
file_name_generated_assessment = file_name + '_GENERATED_ASSESSMENT'
assessment = 'GENERATED ASSESSMENT and PLAN\n\n\nCURRENT MEDICATIONS' + assessment.split('CURRENT MEDICATIONS')[1]
write_to_file(file_name_generated_assessment,patient_note,assessment)
_ = subprocess.call(['open',   file_name_generated_assessment + '.txt'])

Tue Jun 27 at 13:49
Maria_Sanchez.txt_GENERATED_ASSESSMENT was created in /Users/michael/Coding


## Identify the MEDICATIONS - and the relevant MEDICAL CONDITIONS

In [116]:
prompt = f"""

From the note which is delimited by triple backticks, 
extract the names of the medications that the patient is taking and for what medical conditions they are
taking them

Format your response as follows:

CURRENT MEDICATIONS:
1. medication is being taken for ...
2. medication is being taken for ...

N. medication is being taken for ...

Review: ```{patient_note}```
"""
response = get_completion(prompt)
print(response)

CURRENT MEDICATIONS:
1. Enalapril and Hydralazine are being taken for hypertension
2. Simvastatin is being taken to manage hyperlipidemia


In [118]:
color_code_clinic_note(pt_note,list_patterns)

NameError: name 'pt_note' is not defined

## Highlight the MEDICATIONS in the note

In [90]:
prompt = f"""

From the note which is delimited by triple backticks, 
extract the names of the medications that the patient is taking. 

Return the names of the medications in a list

Review: ```{patient_note}```
"""

response = get_completion(prompt)
print(response)
my_list = ast.literal_eval(response)

In [146]:
patient_note_for_display = color_code_clinic_note(patient_note,list_patterns)


In [148]:
# patient_note_for_display = patient_note
# finds meds and ignores case
for ele in my_list:
    pattern = re.compile(re.escape(ele), re.IGNORECASE)
    match = re.search(pattern, patient_note_for_display)
    
    # print(ele, match, match.span()[0],match.span()[1])
    new_ele = patient_note_for_display[match.span()[0]:match.span()[1]]
    patient_note_for_display = \
    patient_note_for_display.replace(new_ele,'\033[92;1m' + 
                                     new_ele.upper() + '\033[0m')
print(patient_note_for_display)

Patient Information:
Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic
Address : Sacramento CA

Reason for Visit: Annual Physical

History of Present Illness:
The patient, a 67-year-old white female, presents today for her annual physical examination. She has not reported any specific complaints or concerns at this time.

Medical History:
The patient has a history of hypertension, which is well-controlled with ENALAPRIL and HYDRALAZINE
She also has a history of hyperlipidemia and is on a SIMVASTATIN to manage her cholesterol levels.
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.


Family History:
The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Her mother is alive and in good health. Mother had colon cancer but is currently in remission.

Social History:
The patient smokes 1/2 ppd and has a glass of wine most evenings. She is retired and leads

# Preventive Care Recommendations - as Written by Physician  
- Extract relevant information to form recommendations regarding preventive healthcare.
- 3-5 areas appropriate for the patient described in the clinical note.


In [149]:
prompt = f"""
Your task is to extract relevant information from \ 
a clinical note to form recommendations regarding preventive healthcare.


Determine 3-5 areas that would be appropriate for the patient described in the clinical note.

Format your response as follows, with each recommendation separated by a blank line
Preventive Health Recommendations
1. Topic followed by brief description on the next line

2. Topic followed by brief description on the next line

etc


Review: ```{patient_note}```
"""

recommendations = get_completion(prompt)
print(recommendations)

Preventive Health Recommendations

1. Cardiovascular Health: Given the patient's family history of cardiovascular disease and her own history of hypertension and hyperlipidemia, it is recommended that she undergo regular cardiovascular screenings, including blood pressure checks, lipid panels, and electrocardiograms. Additionally, the patient should be advised to maintain a heart-healthy diet, engage in regular physical activity, and quit smoking to reduce her risk of cardiovascular disease.

2. Cancer Screening: The patient's mother had colon cancer, which puts the patient at an increased risk for developing the disease. Therefore, it is recommended that the patient undergo regular colon cancer screenings, such as colonoscopies, to detect any abnormalities early on.

3. Smoking Cessation: The patient currently smokes half a pack of cigarettes per day, which puts her at an increased risk for a variety of health problems, including lung cancer, heart disease, and stroke. Therefore, it i

In [11]:
prompt = f"""
Acting as an interpreter, translate the {recommendations} into Spanish and Vietnamese
is to extract 

Review: ```{recommendations}```
"""
translation = get_completion(prompt)
print(translation)


KeyboardInterrupt



## Show Specific Reasons for Recommendations
- Medical Consideration
    - Recommendation
    - Reason for Recommendation

In [15]:
prompt = f"""
Your task is to extract relevant information from \ 
a clinical note to form recommendations regarding preventive healthcare.


Determine 3-5 areas that would be appropriate for the patient described in the clinical note. 
For each recommendation, Show the parts of the 
clinical note that prompt these recommendations

Format your response as follows, with each recommendation separated by a blank line
Preventive Health Recommendations
1. Topic 
    a. brief description on the next line
    b. Section of note that pertains to recommendation 

2. Topic 
    a. brief description on the next line
    b. Section of note that pertains to recommendation 

etc


Review: ```{patient_note}```
"""

recommendations = get_completion(prompt)
print(recommendations)

Preventive Health Recommendations

1. Cardiovascular Disease Prevention
    a. The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s.
    b. Medical History: The patient has a history of hypertension and hyperlipidemia, both of which are well-managed with medications.
    c. Assessment and Plan: Recommend regular cardiovascular screening, including blood pressure and cholesterol checks, as well as lifestyle modifications such as smoking cessation and regular exercise.

2. Cancer Screening
    a. Family History: The patient's mother had colon cancer but is currently in remission.
    b. Assessment and Plan: Recommend regular colon cancer screening, such as colonoscopy, as well as breast cancer screening, such as mammography.

3. Smoking Cessation
    a. Social History: The patient smokes 1/2 ppd.
    b. Assessment and Plan: Recommend smoking cessation interventions, such as nicotine replacement therapy or counseling, to red

In [16]:
file_name = 'Maria Sanchez_Friday'
write_to_file(file_name,patient_note_cap_version,assessment + '\n\n' + recommendations)
_ = subprocess.call(['open',   file_name + '.txt'])

Mon Jun 26 at 09:48
Maria Sanchez_Friday was created in /Users/michael/Coding


In [21]:
# print(patient_note)

In [22]:
prompt = f"""
Acting as a medical coding expert, extract the patient's name, gender and ethnicity 
and output a dictionary with keys of "name","age", "gender",and "ethnicity"

Review: ```{patient_note}```
"""
patient_details= get_completion(prompt)
print(patient_details)

Output:
{
"name": "Maria Sanchez",
"age": 67,
"gender": "Female",
"ethnicity": "Hispanic"
}


# Preventive Care Recommendations - Personalized for the Patient

In [42]:
prompt = f"""
Your task is to extract relevant information from \ 
a clinical note to form recommendations regarding preventive healthcare.

Determine 3-5 areas that would be appropriate for the patient described in the clinical note.

Format your response as follows, with each recommendation separated by a blank line
Preventive Health Recommendations
1. Topic followed by brief description on the next line

2. Topic followed by brief description on the next line
etc

If a recommendation references smoking cessation,
include the English phone information near where the patient lives which could include any of the following:
Kick it California
•	English:  1-800-300-8086
•	Hispanic:  1-800-600-8191
•	Chinese:  1-800-838-8917
•	Korean:  1-800-556-5564
•	Vietnamese:  1-800-778-8440

Always include the English number but if the patient's name or ethnicity suggests it, please add additional
numbers as appropriate

Use a friendly tone and address the recommendations to the patient, 
using their first name in each recommendation so that 
a personalized message is created

Review: ```{patient_note}```
"""

personalized_recommendations = get_completion(prompt)

patient_name = file_name.split(' ')[0]
personalized_recommendations_green = \
    personalized_recommendations.replace(patient_name,'\033[92;1m' + patient_name.upper() + '\033[0m')
print(personalized_recommendations_green)

Preventive Health Recommendations

1. Smoking cessation: Maria, quitting smoking is one of the best things you can do for your health. I recommend calling Kick it California at 1-800-300-8086 to get started on your journey to a smoke-free life.

2. Cardiovascular disease prevention: Given your family history of cardiovascular disease, it's important to take steps to prevent it. I recommend incorporating more heart-healthy foods into your diet, such as fruits, vegetables, and whole grains. Additionally, regular exercise can help keep your heart healthy.

3. Cancer screening: Maria, given your mother's history of colon cancer, it's important to stay up to date on your cancer screenings. I recommend scheduling a colonoscopy with your primary care physician to ensure that you are in good health.

4. Osteoporosis prevention: As a woman over the age of 50, you are at an increased risk for osteoporosis. I recommend incorporating weight-bearing exercises into your routine, such as walking or l

In [28]:
# patient_name = file_name.split(' ')[0]
# response = \
#     response.replace(patient_name,patient_name.upper())
# # print(response)

file_name = 'Maria Sanchez_Friday'
write_to_file(file_name,patient_note_cap_version,assessment + '\n\n' + personalized_recommendations)
# write_to_file(file_name,patient_note_original,response)
_ = subprocess.call(['open',   file_name + '.txt'])

Sat Jun 24 at 14:09
Maria Sanchez_Friday was created in /Users/michael/Coding


In [14]:
url_chatGPT_usage = 'https://platform.openai.com/account/usage'
webbrowser.open_new(url_chatGPT_usage)
 

True

In [12]:

poor_habits_dict = {
    "heart-healthy" : "https://health.ucdavis.edu/blog/good-food/5-heart-healthy-steps-to-work-into-your-diet/2020/02" ,
    "colonoscopy": "https://health.ucdavis.edu/radiology/specialties/colon.html",
    "osteoporosis" : "https://health.ucdavis.edu/news/headlines/rheumatologist-nancy-lane-discusses-latest-research-on-osteoarthritis-and-osteoporosis/2023/02",
    "smoking": "https://health.ucdavis.edu/cancer/support/support-services/tobacco-cessation.html#stoptobaccoprogram",
    "alcohol": "https://health.ucdavis.edu/medicalcenter/cliniclocations/specialtycare/substance-use-addiction-Sacramento-Midtown.html",
    "depression": "https://health.ucdavis.edu/psychiatry/index.html",
    "eating disorder" : "https://health.ucdavis.edu/blog/cultivating-health/eating-disorders-symptoms-types-risks-and-ways-to-seek-help/2023/02"
}

# Website links to Preventive Care Recommendations

In [13]:
prompt = f"""
Your task is to insert relevant URL web links into  
the recommendations regarding preventive healthcare.

If a recommendation pertains to one of the keys of {poor_habits_dict},
then at the end of the recommendation, 
insert the URL found in the value of the dictionary in the following format:
You can find more information at <URL> at our UC Davis Health System website.

Be sure that key in the dictionary is found in the recommendation before inserting the weblink


Review: ```{personalized_recommendations}```
"""

URLs_inserted_into_personalized_recommendations = get_completion(prompt)
print(URLs_inserted_into_personalized_recommendations)

NameError: name 'personalized_recommendations' is not defined

# Extra

In [ ]:
clinical_note ="""Medical History:

Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic


The patient has a history of hypertension, which is well-controlled with medication (name of medication).
She also has a history of hyperlipidemia and is on a statin medication (name of medication) to manage her cholesterol levels.
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.
Family History:
The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Her mother is alive and in good health. There is no family history of cancer or other major illnesses.

Social History:
The patient is a non-smoker and does not consume alcohol excessively. She is retired and leads a moderately active lifestyle, engaging in regular walking and light exercises. She follows a balanced diet and has no known food allergies.

"""

## otolaryngologist

In [58]:
medical_specialist = 'otolaryngologist'
the_prompt = extract_info_as_a(medical_specialist)
print(the_prompt.split('Review: ')[0])
assessment = get_completion(the_prompt)
print(assessment)


    Acting as a otolaryngologist, your task is to extract relevant information from \ 
    a clinical note to produce an assessent and plan for the patient.


    Format your response as follows:
    Patient Age:
    Patient Gender:
    CURRENT MEDICATIONS:
    1.
    2.

    N.

    MEDICAL CONSIDERATIONS:
    1. Medical Concern
        Assessment : 
        Plan       :

    2. Medical Concern
        Assessment : 
        Plan       :

    N. Medical Concern
        Assessment : 
        Plan       :


    Preventive Care Recommendations:
        a.
        b.

    Preventive Care Recommendations should include any recommendations about 
    screening for cancer  . 
    List any screening recommendations under 'Preventive Care Recommendations' 
    and do not include them as a Medical Consideration.
    Be sure to mention Education regarding alcohol and tobacco in the preventive care recommendations, if applicable
    
    Do not refer patient to a doctor or specialist that is in t